# BERT

Dziś omówimy BERTa -- model, który transformuje nasze tokeny w embeddingi kontekstowe o bardzo dobrej jakości (lepszej niż poprzednie metody, jednak za cenę czasu przetwarzania).

Wykorzystamy dziś bibliotekę `transformers` dlatego zainstalujmy ją najpierw:

In [1]:
!pip install transformers

#BERT

## Zadanie 1 (1 punkt): Tokenizacja przy użyciu algorytmu WordPiece

Poniżej znajduje się prosty fragment kodu, który pobiera model `bert-base` (wersja `uncased` - ten model wprowadził etap wstępnego przetwarzania danych, który przekształcał każdy tekst na tekst pisany małymi literami) i tworzy instancję odpowiedniego tokenizatora dla tego modelu. O tym konkretnym modelu można dowiedzieć się tutaj: https://huggingface.co/bert-base-uncased (Zachęcam do przeczytania opisu! Wiele modeli hostowanych na stronie huggingface ma świetną dokumentację i zawsze warto ją sprawdzić).

Następnie w linii 4 definiujemy tekst do tokenizacji, uruchamiamy tokenizer w linii 5 i używamy tokenizowanych danych wejściowych jako danych wejściowych do modelu BERT, który jest wywoływany w linii 6.

Uruchom poniższy kod. Wygenerowaliśmy osadzania BERT przy użyciu 6 linii kodu! ;)

Jeśli uda sie uruchomić ten kod -- gratulacje, to wystarczy na 1 punkt ;).

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Uruchomienie tokenizatora przez zwykłe wywołanie obiektu tokenizatora (`tokenizer(text, return_tensor='pt')`) zwraca słownik pythonowy zawierający 3 klucze: `input_ids`, `token_type_ids` i `attention_mask`. Na razie skupmy się tylko na `input_ids`.

Możesz odwiedzić stronę internetową: https://huggingface.co/docs/transformers/glossary, aby dowiedzieć się więcej o roli `token_type_ids` i `attention_mask`.


`input_ids` to lista list (reprezentowana jako tensor). Lista zewnętrzna gromadzi dokumenty, podczas gdy listy wewnętrzne gromadzą tokeny w tym dokumencie. Tutaj przetworzyliśmy tylko jeden dokument (zdanie), więc jest tylko jedna „zewnętrzna” lista.

Każda z wewnętrznych list zawiera sekwencję identyfikatorów. To są pozycje tokenów w słowniku. Można ich użyć do generowania reprezentacji one-hot encoding dla naszych słów, ponieważ znając długość słownika i znając pozycję danego tokena w słowniku, możemy wygenerować wektor długości słownika, który jest wypełniony zerami , następnie ustawiamy wartość przypisaną do pozycji tokena na 1, aby wygenerować kodowanie typu one-hot.

Aby wygenerować identyfikatory, musimy najpierw dokonać tokenizacji naszego tekstu.

Te identyfikatory wymagają mniej pamięci niż przechowywanie tokenów jako łańcuchów znaków (np zamiast słowa "najciekawszy" będziemy mieć jedną liczbę, np. 2045)!

Uruchom poniższy kod, aby zobaczyć jakie wyjście wygeneruje tokenizator. Należy pamiętać, że liczba identyfikatorów tokenów wygenerowanych przez tokenizator nie jest równa liczbie tokenów w sekwencji. Za chwilę zobaczymy, dlaczego tak jest.

In [3]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Identyfikatory generowane przez tokenizator są liczbami, które ciężko nam zrozumieć. Jednak ponieważ tokenizator zawiera mapowanie zamieniające tokeny na ich pozycje, możemy odwrócić ten proces.

Pierwszy wiersz kodu pobiera identyfikatory zdefiniowane dla pierwszego zdania.
Następnie wywołujemy `convert_ids_to_tokens`, aby przekształcić te identyfikatory w tokeny. Uruchom kod i przeanalizuj wyjście.

In [4]:
first_sentence_ids = encoded_input['input_ids'][0]
tokenizer.convert_ids_to_tokens(first_sentence_ids)

['[CLS]',
 'replace',
 'me',
 'by',
 'any',
 'text',
 'you',
 "'",
 'd',
 'like',
 '.',
 '[SEP]']

Wow, widzimy, że tokenizator nie tylko tokenizuje nasz tekst (dzieli na tokeny), ale także generuje specjalne tokeny wymagane przez BERT ([SEP] i [CLS])! Należy pamiętać, że po tokenizacji nie mamy wielkich liter w naszych tokenach. Jest to spowodowane użyciem modelu `bert-base-uncased`. Ponieważ model został przeszkolony na danych pisanych małymi literami, tokenizator zapewnia również, że tokeny są zamienione do postaci zawierającej jedynie małe litery.


## Subword units -- Jednostki podrzędne

Jednak długość wygenerowanych identifykatorów -- `input_ids` może być jeszcze większa w stosunku do długości naszego tekstu. Czasami słownik modelu nie zawiera danego słowa w całości. Ponieważ BERT używał tokenizacji WordPiece, do obsługi takich przypadków tokenizator próbuje podzielić te słowa na mniejsze fragmenty, które są przechowywane w naszym słowniku. Wykonajmy tokenizację dokumentu zawierającego rzadkie słowa i zobaczmy jaki wynik otrzymamy.

In [5]:
tokenizer_output = tokenizer(['NVIDIA DGX A100'])

input_ids = tokenizer_output['input_ids'][0]
input_ids

[101, 1050, 17258, 2401, 1040, 2290, 2595, 17350, 8889, 102]

Otrzymaliśmy całkiem sporo tokenów jak na tak krotki tekst! Sprawdźmy co one reprezentują:

In [6]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', 'n', '##vid', '##ia', 'd', '##g', '##x', 'a1', '##00', '[SEP]']

Jak widzimy, słownik przypisany do `bert-base` nie zawiera tokenów takich jak "nvidia", "dgx" i "a100", dlatego są one podzielone na jednostki podrzędne (subword units).

Za każdym razem, gdy dany subword unit rozpoczyna się podwójnym haszem (##), wiemy, że jest on kontynuacją poprzedniego tokenu (subword unitu).

Możemy użyć tych informacji do zrekonstruowania oryginalnego tekstu, łącząc te jednostki podrzędne (czasami nazywane subtokenami) w pełne tokeny. Możemy osiągnąć ten cel za pomocą następującego wiersza kodu:

In [7]:
tokenizer.decode(input_ids)

'[CLS] nvidia dgx a100 [SEP]'

Jeśli wiemy, jak mapować tokeny do ich pozycji w słowniku, jedyną brakującą częścią jest określenie, jak długi powinien być nasz wektor one-hot-encoding (lub jak duży jest nasz słownik).

To przekształcenie identyfikatorów w kodowanie one-hot jest wykonywane automatycznie przez bibliotekę `transformer`. Możesz jednak łatwo sprawdzić rozmiar słownika, używając następującego wiersza kodu:

In [8]:
tokenizer.vocab_size

30522

## Zadanie 2 (4 punkty): Wykorzystanie wstępnie przeszkolonego BERT do wygenerowania cech, które można wykorzystać do rozwiązania zadania klasyfikacyjnego.

Jak omówiliśmy podczas wykładu, możemy wygenerować wektor o stałej długości reprezentujący dowolne wejście, biorąc jedynie embedding utworzony dla tokena `[CLS]` (który jest reprezentacją całej sekwencji).

W tym zadaniu Twoim celem jest użycie wstępnie wytrenowanego modelu BERT w celu uzyskania reprezentacji dla danego zestawu danych. Następnie reprezentacje te zostaną użyte do trenowania modelu regresji logistycznej.

Postaramy się wygenerować rozwiązanie, które wykryje, czy dana recenzja jest pozytywna, czy nie!

W tym celu użyjemy wariantu BERT o nazwie `distilBERT`. DistilBERT to mniejszy model BERT zachowujący prawie pełną jakość oryginalnego BERTa. W rzeczywistości jest to skompresowany model BERT. Zachowuje się tak samo, ale jest destylowany, więc naszym celem było osiągnięcie podobnej jakości przy mniejszych parametrach.

Postępuj zgodnie z samouczkiem, który znajdziesz tutaj: https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb (jest nawet post na blogu dotyczący ten samouczek: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)


Umieść fragmenty kodu z tego tutorialu tu i obserwuj wyniki. Jeśli uda ci się przejść przez ten tutorial - otrzymasz 4 punkty.

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

batch_1 = df[:2000]

batch_1[1].value_counts()

# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()

labels = batch_1[1]

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

lr_clf.score(test_features, test_labels)

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

best parameters:  {'C': 5.263252631578947}
best scrores:  0.8280000000000001
Dummy classifier score: 0.538 (+/- 0.00)


## Zadanie 3 (opcjonalne — niewymagane): dostrajanie BERT

W powyższym przykładzie model BERT (a dokładniej jego mniejszy członek rodziny: distilBERT) został użyty tylko do dostarczenia wektorów reprezentujących całe dokumenty.

Teraz chcielibyśmy dostroić istniejący model BERT. Jak omówiliśmy podczas wykładu, możemy to osiągnąć, po prostu przełączając górną warstwę sieci. Zamiast rozwiązywać zadania Masked Language Model i Next Sentence Prediction, możemy dodać własną warstwę klasyfikacyjną (nazywaną też głową klasyfikacji) i wyszkolić całą naszą sieć do rozwiązania danego zadania.

Jest świetny i łatwy do naśladowania samouczek dotyczący dostrajania, dostępny tutaj: https://github.com/huggingface/notebooks/blob/main/transformers_doc/training.ipynb

Jeśli chcesz, skorzystaj z tego samouczka (możesz skopiować i wkleić fragmenty kodu z notatnika tutaj).

In [1]:
# Transformers installation
! pip install transformers datasets

from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

!pip install transformers[torch]
!pip install accelerate -U
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

del model
del pytorch_model
del trainer
torch.cuda.empty_cache()

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`